In [1]:
# Copyright 2021 Google
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [ ]:
!pip install cirq --pre

In [3]:
import cirq
import itertools
import numpy as np
import matplotlib.pyplot as plt
import recirq.time_crystals as time_crystals

In [4]:
# define the qubits we'll be using
qubit_locations = [(3, 9), (3, 8), (3, 7), (4, 7), (4, 8), (5, 8), (5, 7), (5, 6), (6, 6), (6, 5), (7, 5), (8, 5),
              (8, 4), (8, 3), (7, 3), (6, 3)]

qubits = [cirq.GridQubit(*idx) for idx in qubit_locations]
num_qubits = len(qubits)

num_cycles = 100

# dir to store data in
base_dir = time_crystals.DEFAULT_BASE_DIR

In [5]:
%%time
# Figure 2d

# number of disorder instances to average over
disorder_instances = 36

# disorder instances h
local_fields = np.random.uniform(-1.0, 1.0, (disorder_instances, num_qubits))

# initial states, one for each disorder instance
initial_states = np.random.choice(2, (disorder_instances, num_qubits))

# gammas for phased FSim gates
gammas = np.random.uniform(-0.5*np.pi, -1.5*np.pi, (disorder_instances, num_qubits - 1))

# create comparison task
options_dict = {
    'g': [0.6, 0.94],
    'initial_states': [initial_states], 
    'local_fields' : [local_fields],
    'gammas': [gammas]
}
comparedtctask = time_crystals.CompareDTCTask(qubits, num_cycles, disorder_instances, options_dict)

# collect average polarizations by g option
average_polarizations = np.empty((2, num_cycles+1, num_qubits))

# create polarizations generator
polarizations_generator = time_crystals.run_comparison_experiment(comparedtctask, autocorrelate=True, take_abs=False)

for index, disorder_averaged_polarizations in enumerate(polarizations_generator):
    average_polarizations[index, :, :] = disorder_averaged_polarizations

# save data in json format
filename = f'{base_dir}/2d.json'
with open(filename, 'w+') as f:
    cirq.to_json(average_polarizations, file_or_fn=f)

CPU times: user 2min 48s, sys: 84.9 ms, total: 2min 48s
Wall time: 2min 48s


In [6]:
%%time
# Figure 3a

# number of disorder instances to average over
disorder_instances = 24

# disorder instances h
local_fields = np.random.uniform(-1.0, 1.0, (disorder_instances, num_qubits))

# prepare 3 initial states to compare
neel_initial_state = np.tile([0,1], num_qubits//2)
polarized_initial_state = np.full(num_qubits, 0)
random_initial_state = [0,0,1,1,1,0,0,0,0,1,0,0,1,1,0,0,1,1,1,1][:num_qubits]
initial_states = [neel_initial_state, polarized_initial_state, random_initial_state]

# prepare random and fixed phis to compare
disordered_phis = np.random.uniform(-1.5*np.pi, -0.5*np.pi, (disorder_instances, num_qubits - 1))
fixed_phis = np.full((disorder_instances, num_qubits - 1), -0.4)

# create comparison task
options_dict = {
    'local_fields': [local_fields], 
    'initial_state': initial_states,
    'phis': [disordered_phis, fixed_phis]
}
options_order = ['local_fields', 'phis', 'initial_state']
comparedtctask = time_crystals.CompareDTCTask(qubits, num_cycles, disorder_instances, options_dict, options_order)

# prepare polarizations and indices generators
polarizations_generator = time_crystals.run_comparison_experiment(comparedtctask, autocorrelate=True, take_abs=False)
indices_iterator = itertools.product(range(2), range(len(initial_states)))

# collect polarizations averaged over qubit sites by phi option and initial state
average_polarizations = np.empty((2, len(initial_states), num_cycles+1))

for (phi_index, initial_state_index), disorder_averaged_polarizations in zip(indices_iterator, polarizations_generator):
    # store average over all qubit sites
    average_polarizations[phi_index, initial_state_index, :] = np.mean(disorder_averaged_polarizations, axis=1)

# save data in json format
filename = f'{base_dir}/3a.json'
with open(filename, 'w+') as f:
    cirq.to_json(average_polarizations, file_or_fn=f)

CPU times: user 5min 37s, sys: 140 ms, total: 5min 37s
Wall time: 5min 37s


In [7]:
%%time
# Figure 3b

# instances for disorder and random initial states
disorder_instances = 24
initial_state_instances = 20# is 500 in the paper

# disorder instances h
local_fields = np.random.uniform(-1.0, 1.0, (disorder_instances, num_qubits))

# prepare random initial states
initial_states = np.random.choice(2, (initial_state_instances, num_qubits))

# Different phis to compare
disordered_phis = np.random.uniform(-1.5*np.pi, -0.5*np.pi, (disorder_instances, num_qubits - 1))
fixed_phis = np.full((disorder_instances, num_qubits - 1), -0.4)

# create comparison task
options_dict = {
    'local_fields': [local_fields],
    'initial_state': initial_states,
    'phis': [disordered_phis, fixed_phis]
}
options_order = ['local_fields', 'phis', 'initial_state']
comparedtctask = time_crystals.CompareDTCTask(qubits, num_cycles, disorder_instances, options_dict, options_order)

# prepare polarizations and indices generators
polarizations_generator = time_crystals.run_comparison_experiment(comparedtctask, autocorrelate=True, take_abs=True)
indices_iterator = itertools.product(range(2), range(len(initial_states)))

# collect average polarizations by phi option and initial state
average_polarizations = np.empty((2, initial_state_instances))

for (phi_index, initial_state_index), disorder_averaged_polarizations in zip(indices_iterator, polarizations_generator):
    # store average over qubit sites and cycles 30 and 31
    average_polarizations[phi_index, initial_state_index] = np.mean(disorder_averaged_polarizations[30:32, :])
    
# save data in json format
filename = f'{base_dir}/3b.json'
with open(filename, 'w+') as f:
    cirq.to_json(average_polarizations, file_or_fn=f)

CPU times: user 37min 47s, sys: 925 ms, total: 37min 48s
Wall time: 37min 48s


In [8]:
%%time
# Figure 3c

# number of disorder instances to average over
disorder_instances = 24

# disorder parameters h
local_fields = np.random.uniform(-1.0, 1.0, (disorder_instances, num_qubits))

# disordered and fixed phis for all instances and qubits
disordered_phis = np.random.uniform(-1.5*np.pi, -0.5*np.pi, (disorder_instances, num_qubits - 1))
fixed_phis = np.full((disorder_instances, num_qubits - 1), -0.4)

# prepare two initial states to compare
polarized_initial_state = [0]*num_qubits
disturb_qubit = 11
disturbed_polarized_initial_state = list(polarized_initial_state)
disturbed_polarized_initial_state[disturb_qubit] = 1
initial_states = [polarized_initial_state, disturbed_polarized_initial_state]

# create comparison task
options_dict = {
    'local_fields': [local_fields],
    'initial_state': initial_states,
    'phis': [disordered_phis, fixed_phis]
    
}
options_order = ['local_fields', 'phis', 'initial_state']
comparedtctask = time_crystals.CompareDTCTask(qubits, num_cycles, disorder_instances, options_dict, options_order)

# prepare polarizations and indices generators
polarizations_generator = time_crystals.run_comparison_experiment(comparedtctask, autocorrelate=False, take_abs=False)
indices_iterator = itertools.product(range(2), range(len(initial_states)))

# collect average polarizations by phi option, initial_state over cycles 30 through 60 and qubits 11 through 14
average_polarizations = np.empty((2, 2, 31, 4))

for (phi_index, initial_state_index), disorder_averaged_polarizations in zip(indices_iterator, polarizations_generator):
    # store only some cycles and qubits
    average_polarizations[phi_index, initial_state_index, :, :] = disorder_averaged_polarizations[30:61, 11:15]

# save data in json format
filename = f'{base_dir}/3c.json'
with open(filename, 'w+') as f:
    cirq.to_json(average_polarizations, file_or_fn=f)

CPU times: user 3min 44s, sys: 108 ms, total: 3min 44s
Wall time: 3min 44s


In [9]:
%%time
# Figure 3d

# prepare two initial states to compare
polarized_initial_state = np.full(num_qubits, 0)
disturb_qubit = 11
disturbed_polarized_initial_state = polarized_initial_state.copy()
disturbed_polarized_initial_state[disturb_qubit] = 1
initial_states = [polarized_initial_state, disturbed_polarized_initial_state]

# we have different disorder instances for the two initial states
disorder_instances_options = [64, 81]

# collect polarizations first
average_polarizations = np.empty((2, 2, num_cycles + 1, num_qubits))

# iterate over initial states and their associated number of disorder instances
for initial_state_index, (initial_state, disorder_instances) in enumerate(zip(initial_states, disorder_instances_options)): 
    
    # disorder parameter h
    local_fields = np.random.uniform(-1.0, 1.0, (disorder_instances, num_qubits))
    
    # expand disordered and fixed phis for all instances and qubits
    disordered_phis = np.random.uniform(-1.5*np.pi, -0.5*np.pi, (disorder_instances, num_qubits - 1))
    fixed_phis = np.full((disorder_instances, num_qubits - 1), -0.4)

    # create comparison task
    options_dict = {
        'initial_state': [initial_state],
        'local_fields': [local_fields],
        'phis': [disordered_phis, fixed_phis]
    }
    options_order = ['local_fields', 'phis', 'initial_state']
    comparedtctask = time_crystals.CompareDTCTask(qubits, num_cycles, disorder_instances, options_dict, options_order)

    # prepare polarizations and indices generators
    polarizations_generator = time_crystals.run_comparison_experiment(comparedtctask, autocorrelate=False, take_abs=False)
    
    for phi_index, disorder_averaged_polarizations in enumerate(polarizations_generator):
        # store average polarizations
        average_polarizations[phi_index, initial_state_index, :, :] = disorder_averaged_polarizations

        
# save data in json format
filename = f'{base_dir}/3d.json'
with open(filename, 'w+') as f:
    cirq.to_json(average_polarizations, file_or_fn=f)

CPU times: user 11min 25s, sys: 272 ms, total: 11min 25s
Wall time: 11min 25s
